In [1]:
!pip install roboflow ultralytics


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 85.3/85.3 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 66.8/66.8 kB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.9/49.9 MB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 35.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 76.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 32.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 29.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 12.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127

In [2]:
from roboflow import Roboflow

rf = Roboflow(api_key="")  #<------------------------- Replace with your API key

project1 = rf.workspace("nfc-x5ixs").project("soil-classification-cy3ss")
dataset1 = project1.version(1).download("yolov8")

project2 = rf.workspace("jullajak").project("soils-hfu4w")
dataset2 = project2.version(1).download("yolov8")


loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soil-Classification-1 in yolov8:: 100%|██████████| 2980/2980 [00:00<00:00, 4709.86it/s]


Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.
loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Soils-1 in yolov8:: 100%|██████████| 412/412 [00:00<00:00, 6180.23it/s]


In [3]:
import os

# Map old class IDs (0:Clay, 1:Loam, 2:Sand) → new IDs
mapping = {0: 2, 1: 4, 2: 5}

for split in ['train', 'valid', 'test']:
    label_path = os.path.join(dataset2.location, split, "labels")
    if os.path.exists(label_path):
        for file_name in os.listdir(label_path):
            file_path = os.path.join(label_path, file_name)
            with open(file_path, "r") as f:
                lines = f.readlines()
            with open(file_path, "w") as f:
                for line in lines:
                    parts = line.strip().split()
                    if parts:
                        class_id = int(parts[0])
                        if class_id in mapping:
                            parts[0] = str(mapping[class_id])
                            f.write(" ".join(parts) + "\n")


In [4]:
import shutil

# Create folder structure
for split in ["train", "valid", "test"]:
    os.makedirs(f"/content/merged_dataset/images/{split}", exist_ok=True)
    os.makedirs(f"/content/merged_dataset/labels/{split}", exist_ok=True)

# Copy from dataset1
for split in ["train", "valid", "test"]:
    src_img = os.path.join(dataset1.location, split, "images")
    src_lbl = os.path.join(dataset1.location, split, "labels")
    if os.path.exists(src_img):
        shutil.copytree(src_img, f"/content/merged_dataset/images/{split}", dirs_exist_ok=True)
    if os.path.exists(src_lbl):
        shutil.copytree(src_lbl, f"/content/merged_dataset/labels/{split}", dirs_exist_ok=True)

# Copy from dataset2 (already fixed labels)
for split in ["train", "valid", "test"]:
    src_img = os.path.join(dataset2.location, split, "images")
    src_lbl = os.path.join(dataset2.location, split, "labels")
    if os.path.exists(src_img):
        shutil.copytree(src_img, f"/content/merged_dataset/images/{split}", dirs_exist_ok=True)
    if os.path.exists(src_lbl):
        shutil.copytree(src_lbl, f"/content/merged_dataset/labels/{split}", dirs_exist_ok=True)


In [5]:
yaml_content = """
train: /content/merged_dataset/images/train
val: /content/merged_dataset/images/valid
test: /content/merged_dataset/images/test

nc: 6
names: ['Alluvial', 'Black', 'Clay', 'Red', 'Loam', 'Sand']
"""

with open("/content/merged_dataset/data.yaml", "w") as f:
    f.write(yaml_content)


In [6]:
from ultralytics import YOLO

model = YOLO("yolov8n.yaml")

model.train(data="/content/merged_dataset/data.yaml", epochs=50, imgsz=640, batch=16)


Ultralytics 8.3.120 🚀 Python-3.11.12 torch-2.6.0+cu124 CPU (Intel Xeon 2.20GHz)
engine/trainer: task=detect, mode=train, model=yolov8n.yaml, data=/content/merged_dataset/data.yaml, epochs=50, time=None, patience=100, batch=16, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_

100%|██████████| 755k/755k [00:00<00:00, 10.3MB/s]

Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      7360  ultralytics.nn.modules.block.C2f             [32, 32, 1, True]             
  3                  -1  1     18560  ultralytics.nn.modules.conv.Conv             [32, 64, 3, 2]                
  4                  -1  2     49664  ultralytics.nn.modules.block.C2f             [64, 64, 2, True]             
  5                  -1  1     73984  ultralytics.nn.modules.conv.Conv             [64, 128, 3, 2]               
  6                  -1  2    197632  ultralytics.nn.modules.block.C2f             [128, 128, 2, True]           
  7                  -1  1    295424  ultralytics

 22        [15, 18, 21]  1    752482  ultralytics.nn.modules.head.Detect           [6, [64, 128, 256]]           
YOLOv8n summary: 129 layers, 3,012,018 parameters, 3,012,002 gradients, 8.2 GFLOPs

Freezing layer 'model.22.dfl.conv.weight'
train: Fast image access ✅ (ping: 0.0±0.0 ms, read: 954.9±565.1 MB/s, size: 80.2 KB)


train: Scanning /content/merged_dataset/labels/train... 1434 images, 0 backgrounds, 0 corrupt: 100%|██████████| 1434/1434 [00:00<00:00, 1579.18it/s]

...

val: Scanning /content/merged_dataset/labels/valid... 167 images, 0 backgrounds, 0 corrupt: 100%|██████████| 167/167 [00:00<00:00, 1861.71it/s]

val: New cache created: /content/merged_dataset/labels/valid.cache
WARNING ⚠️ Box and segment counts should be equal, but got len(segments) = 2, len(boxes) = 203. To resolve this only boxes will be used and all segments will be removed. To avoid this please supply either a detect or segment dataset, not a detect-segment mixed dataset.


Plotting labels to runs/detect/train/labels.jpg... 
optimizer: 'optimizer=auto' found, ignoring 'lr0=0.01' and 'momentum=0.937' and determining best 'optimizer', 'lr0' and 'momentum' automatically... 
optimizer: AdamW(lr=0.001, momentum=0.9) with parameter groups 57 weight(decay=0.0), 64 weight(decay=0.0005), 63 bias(decay=0.0)
Image sizes 640 train, 640 val
Using 0 dataloader workers
Logging results to runs/detect/train
Starting training for 50 epochs...

      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


       1/50         0G      3.294      4.256      4.187         46        640:   1%|          | 1/90 [00:33<49:18, 33.24s/it]


KeyboardInterrupt: 